In [1]:
import os
import subprocess
import sys

# append path
sys.path.append("/wynton/home/ahituv/fongsl/tools/py_/")

# import config reader
import config_readwrite as crw

config_tag = sys.argv[1]

config, configfile_name = crw.read_config(os.path.join(os.getcwd(), config_tag))

In [ ]:
DATAPATH = config["DATA"]["PATH"]

# Generate LINSIGHT resources
LINSIGHT_BW_URL = "http://compgen.cshl.edu/LINSIGHT/LINSIGHT.bw"  # precomputed scores. In hg19. 
LINSIGHT_PATH = os.path.join(DATAPATH, "linsight")
LINSIGHT_BW = os.path.join(LINSIGHT_PATH, "LINSIGHT.bw")

# if the datafolder does not exist, make it
if os.path.exists(LINSIGHT_PATH) is False:
    os.mkdir(LINSIGHT_PATH)
    print("made LINSIGHT data folder")

# make a section in config file for LINSIGHT. 
crw.check_section(config, "LINSIGHT")


# download linsight file
if os.pathexists(LINSIGHT_BW) is False:
    print("downloading LINSIGHT.bw (hg19!) from internet")
    os.system(f"wget -O {LINSIGHT_BW} {LINSIGHT_BW_URL}")

# do liftover hg19 --> hg38